In [49]:
import cv2
import mediapipe as mp
import D_Ham
import numpy as np
import os
import D_HistoryPoint as His

In [50]:
hisp = His.His_HPC()
kh = D_Ham.ReadLb_H('./model/Point_History/Label.csv')
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [51]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [52]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [53]:
def make_pose(results, c_lm):
    c_lm.append(results.pose_landmarks.landmark[9].x)
    c_lm.append(results.pose_landmarks.landmark[9].y)

    c_lm.append(results.pose_landmarks.landmark[10].x)
    c_lm.append(results.pose_landmarks.landmark[10].y)
    
    return

In [54]:
def make_l_hand(results, c_lm):
    for id, lm in enumerate(results.left_hand_landmarks.landmark):
        c_lm.append(lm.x)
        c_lm.append(lm.y)
    return

def make_r_hand(results, c_lm):
    for id, lm in enumerate(results.right_hand_landmarks.landmark):
        c_lm.append(lm.x)
        c_lm.append(lm.y)
    return

In [63]:
cap = cv2.VideoCapture(0)
TIME_STEP = 10
lm_list = []
t = []
kt = True
text = ''
dem = 0
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        ret, frame = cap.read()
        # frame = cv2.medianBlur(frame,5)
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        if kt == True:
            if results.pose_landmarks:
                if results.left_hand_landmarks or results.right_hand_landmarks:
                    lm = []
                    make_pose(results, lm)
                    if results.left_hand_landmarks:
                        make_l_hand(results, lm)
                    else:
                        for j in range(42):
                            lm.append(0)
                    if results.right_hand_landmarks:
                        make_r_hand(results, lm)
                    else:
                        for j in range(42):
                            lm.append(0)
                    lm_f = D_Ham.FindPoint_H(frame, lm)
                    lm_pre = D_Ham.pre_process_H(lm_f)
                    lm_list.append(lm_pre)
                    if len(lm_list) == TIME_STEP:
                        lm_save = []
                        lm_save.append(lm_list[0])
                        for s in range(14):
                            lm_save.append(lm_list[int((s + 1) * len(lm_list)/16)])
                        lm_save.append(lm_list[len(lm_list) - 1])

                        lm_his = D_Ham.pre_convert(lm_save)
                        
                        sign_id, val = hisp(lm_his)
                        if (val[sign_id] >= 0.5):
                            text = text + kh[sign_id] + ' '
                        lm_list = []
                        kt = False
                        
                        print(str(sign_id) + "   " + str(val[sign_id]))
        else:
            dem += 1
            if dem == 5:
                dem = 0
                kt = True
                print("True")

        cv2.rectangle(image, (10, 10), (630, 55), (0, 207, 0), -1)
        cv2.putText(image, text, (20, 45),
                                   cv2.FONT_HERSHEY_SIMPLEX, 1.3, (245, 0, 0), 3, cv2.LINE_AA)
        cv2.imshow("Image", image)
        key = cv2.waitKey(10)
        if key == 27:
            break
        elif key == 8:
            lm_list = []
            text = ''
        elif key == 48 or key == 49 or key == 50 or key == 51:
            D_Ham.SavePoint_H(chr(key), lm_list)
            lm_list = []
        
cap.release()
cv2.destroyAllWindows()

48   0.99396366
True
9   0.90987235
True
2   0.9095524
True
23   0.9999994
True
25   0.9952306
True
32   0.7514068
True


In [56]:
cap.release()
cv2.destroyAllWindows()